# Churn Prediction

### importing relevent libraries

In [138]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn import model_selection
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

read the churn csv file from the current working dicrectory

In [260]:
raw_train =  pd.read_csv('churn.csv')
raw_test =  pd.read_csv('test_set.csv')

making a copy of the original dataset

In [293]:
train = raw_train.copy()
test  = raw_test.copy() 

In [262]:
train.head()

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
0,KS,128,area_code_415,no,yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,no
1,OH,107,area_code_415,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,no
2,NJ,137,area_code_415,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,no
3,OH,84,area_code_408,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,no
4,OK,75,area_code_415,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,no


In [294]:
test = test.drop(['row_id','state','Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22', 'Unnamed: 23','area_code'],axis=1)
train = train.drop(['state','area_code'],axis=1)

In [234]:
train.columns

Index(['state', 'account_length', 'area_code', 'international_plan',
       'voice_mail_plan', 'number_vmail_messages', 'total_day_minutes',
       'total_day_calls', 'total_day_charge', 'total_eve_minutes',
       'total_eve_calls', 'total_eve_charge', 'total_night_minutes',
       'total_night_calls', 'total_night_charge', 'total_intl_minutes',
       'total_intl_calls', 'total_intl_charge',
       'number_customer_service_calls', 'churn'],
      dtype='object')

# Train feature engineering

In [295]:
train['international_plan'] = train['international_plan'].map({'yes': 1, 'no': 0})
train['voice_mail_plan'] = train['voice_mail_plan'].map({'yes': 1, 'no': 0})
train['churn'] = train['churn'].map({'yes': 1, 'no': 0})

combining Daily Minutes

In [226]:
train['Daily_Minutes'] = train['total_day_minutes'] + train['total_eve_minutes'] + train['total_night_minutes'] +train['total_intl_minutes']
train =  train.drop(['total_day_minutes','total_eve_minutes','total_night_minutes','total_intl_minutes'], axis=1)

In [105]:
train.columns

Index(['state', 'account_length', 'area_code', 'international_plan',
       'voice_mail_plan', 'number_vmail_messages', 'total_day_calls',
       'total_day_charge', 'total_eve_calls', 'total_eve_charge',
       'total_night_calls', 'total_night_charge', 'total_intl_calls',
       'total_intl_charge', 'number_customer_service_calls', 'churn',
       'Daily_Minutes'],
      dtype='object')

Combining Daily charges

In [177]:
train['Daily_Charge'] = train['total_intl_charge'] + train['total_night_charge'] + train['total_eve_charge'] + train['total_day_charge']                                                                                                                   

In [178]:
train  = train.drop(['total_intl_charge','total_night_charge','total_eve_charge','total_day_charge'], axis=1)

Combinig  Daily calls

In [179]:
train['Service_Calls'] = train['total_night_calls'] + train['total_eve_calls'] + train['total_day_calls']
train = train.drop(['total_night_calls','total_eve_calls','total_day_calls'],axis=1)


Rearrange the columns

# test feature engineering

test dataset preprocessing

In [296]:
test['international_plan'] = test['international_plan'].map({'yes': 1, 'no': 0})
test['voice_mail_plan'] = test['voice_mail_plan'].map({'yes': 1, 'no': 0})

Combining Daily Minutes

In [181]:
test['Daily_Minutes'] = test['total_day_minutes'] + test['total_eve_minutes'] + test['total_night_minutes'] +test['total_intl_minutes']
test =  test.drop(['total_day_minutes','total_eve_minutes','total_night_minutes','total_intl_minutes'], axis=1)

Combining Daily Charges

In [182]:
test['Daily_Charge'] = test['total_intl_charge'] + test['total_night_charge'] + test['total_eve_charge'] + test['total_day_charge'] 
test  = test.drop(['total_intl_charge','total_night_charge','total_eve_charge','total_day_charge'], axis=1)

Combining Service calls

In [183]:
test['Service_Calls'] = test['total_night_calls'] + test['total_eve_calls'] + test['total_day_calls']
test = test.drop(['total_night_calls','total_eve_calls','total_day_calls'],axis=1)

Re arranging columns

In [192]:
test = test.drop(['row_id'],1)

In [188]:
test =  test.drop(['Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22', 'Unnamed: 23'], axis=1)

In [ ]:
list1 = ['state', 'account_length', 'area_code', 'international_plan',
       'voice_mail_plan', 'number_vmail_messages', 'total_intl_calls',
       'number_customer_service_calls', 'churn', 'Daily_Minutes',
       'Daily_Charge', 'Service_Calls

In [298]:
test.shape

(1000, 17)

In [299]:
X.shape

(4000, 17)

# Create target variables

In [297]:
#precited variable
Y = train['churn']

#concatinate the dataset
"""data = pd.concat([train.drop(['churn'], axis=1), test.drop(['row_id','Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22', 'Unnamed: 23'])])"""
X = train.drop(['churn'], axis=1)

# Custom Scaler 

In [300]:
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin

class CustomScaler(BaseEstimator,TransformerMixin): 
    
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        self.scaler = StandardScaler(copy,with_mean,with_std)
        self.columns = columns
        self.mean_ = None
        self.var_ = None

    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.array(np.mean(X[self.columns]))
        self.var_ = np.array(np.var(X[self.columns]))
        return self

    def transform(self, X, y=None, copy=None):
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]
    

In [254]:
train.columns

Index(['account_length', 'area_code', 'international_plan', 'voice_mail_plan',
       'number_vmail_messages', 'total_day_minutes', 'total_day_calls',
       'total_day_charge', 'total_eve_minutes', 'total_eve_calls',
       'total_eve_charge', 'total_night_minutes', 'total_night_calls',
       'total_night_charge', 'total_intl_minutes', 'total_intl_calls',
       'total_intl_charge', 'number_customer_service_calls', 'churn'],
      dtype='object')

In [301]:
columns_to_scale = ['account_length',
       'number_vmail_messages', 'total_day_minutes', 'total_day_calls',
       'total_day_charge', 'total_eve_minutes', 'total_eve_calls',
       'total_eve_charge', 'total_night_minutes', 'total_night_calls',
       'total_night_charge', 'total_intl_minutes', 'total_intl_calls',
       'total_intl_charge', 'number_customer_service_calls']

In [302]:
scaler = CustomScaler(columns_to_scale)

In [303]:
scaler

CustomScaler(columns=['account_length', 'number_vmail_messages', 'total_day_minutes', 'total_day_calls', 'total_day_charge', 'total_eve_minutes', 'total_eve_calls', 'total_eve_charge', 'total_night_minutes', 'total_night_calls', 'total_night_charge', 'total_intl_minutes', 'total_intl_calls', 'total_intl_charge', 'number_customer_service_calls'],
       copy=None, with_mean=None, with_std=None)

In [304]:
scaler.fit(X)
scaler.fit(test)

C:\Users\student-cpt-321\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\student-cpt-321\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


CustomScaler(columns=['account_length', 'number_vmail_messages', 'total_day_minutes', 'total_day_calls', 'total_day_charge', 'total_eve_minutes', 'total_eve_calls', 'total_eve_charge', 'total_night_minutes', 'total_night_calls', 'total_night_charge', 'total_intl_minutes', 'total_intl_calls', 'total_intl_charge', 'number_customer_service_calls'],
       copy=None, with_mean=None, with_std=None)

In [305]:
X = scaler.transform(X)
test_1 = scaler.transform(test)

C:\Users\student-cpt-321\anaconda3\lib\site-packages\ipykernel_launcher.py:20: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
C:\Users\student-cpt-321\anaconda3\lib\site-packages\ipykernel_launcher.py:20: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.


### Get the dummy variables

In [306]:
#X = pd.get_dummies(X, drop_first=True)
#test_1 = pd.get_dummies(X, drop_first=True)
test_1.head()

,account_length,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls
0,0.527999,0,0,-0.505001,0.315390,0.621426,0.314873,-0.631155,-1.137407,-0.630879,0.244133,-0.714668,0.243599,-2.223496,-0.966995,-2.229644,-0.478457
1,-1.096075,0,1,1.647209,0.705186,-0.058014,0.705352,0.915143,0.993709,0.915799,-1.423846,-0.326946,-1.421730,-1.008792,1.514600,-1.002640,-0.478457
2,-0.946160,1,0,-0.505001,-1.614295,-0.423866,-1.614817,0.865646,-0.376294,0.864553,0.751779,0.836219,0.753942,-0.051754,-0.966995,-0.048303,-0.478457
3,0.328113,0,1,1.262886,-1.797615,-0.423866,-1.797570,0.362752,0.892227,0.363746,-0.495176,-0.569272,-0.495055,0.758048,-0.139797,0.756066,-0.478457
4,0.527999,0,1,1.954668,0.106984,-1.260099,0.107148,0.137044,-0.427035,0.137801,-0.948431,-0.908529,-0.947201,0.684430,-0.966995,0.687899,-0.478457


# split the data

In [307]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2, random_state=100)
X_test.head()

,account_length,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls
3218,0.153213,0,1,0.878563,0.024007,-1.730480,0.024285,0.269697,-1.797037,0.270573,-0.094297,0.448497,-0.092153,1.162949,1.928199,1.165068,0.287074
883,-1.395903,0,0,-0.505001,-0.666820,-0.789718,-0.666998,1.107193,0.638523,1.106804,-3.075709,0.884684,-3.073629,-0.824747,3.168997,-0.825406,-0.478457
367,-1.320946,0,0,-0.505001,-1.980935,1.457659,-1.981458,1.057695,0.384819,1.057888,1.154673,0.060775,1.156844,2.856172,-0.553396,2.855607,-0.478457
2226,0.552984,0,0,-0.505001,-0.840492,0.046515,-0.840671,0.241979,1.450376,0.242621,-0.906128,-0.326946,-0.906911,0.095483,-0.966995,0.101664,-0.478457
2578,2.401929,0,0,-0.505001,0.240132,0.360103,0.239956,-0.613336,2.515934,-0.612244,0.554361,1.127010,0.552491,0.574002,-0.139797,0.578832,-1.243988


In [308]:
models = []
models.append(('LR', LogisticRegression(C=0.1)))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('RF', RandomForestClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

In [280]:
## print the model accuracy

In [309]:

results = []
names = []
for name,model in models:
    kfold = model_selection.KFold(n_splits=10)
    cv_result = model_selection.cross_val_score(model,X_train,Y_train, cv = kfold, scoring = "accuracy")
    names.append(name)
    results.append(cv_result)
for i in range(len(names)):
    print(names[i],results[i].mean())

C:\Users\student-cpt-321\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\student-cpt-321\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\student-cpt-321\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\student-cpt-321\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\student-cpt-321\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed

LR 0.859375
LDA 0.8534375000000001
KNN 0.8931250000000001
CART 0.9075
RF 0.9468749999999998
NB 0.86625
SVM 0.914375


C:\Users\student-cpt-321\anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [310]:
eval_set=[(X_test,Y_test)]
my_model = XGBClassifier(n_estimators=500,learning_rate=0.01, n_jobs=4)
my_model.fit(X_train, Y_train,early_stopping_rounds=1000,eval_set=eval_set,verbose = True,eval_metric="error")

[0]	validation_0-error:0.08375
Will train until validation_0-error hasn't improved in 1000 rounds.
[1]	validation_0-error:0.08375
[2]	validation_0-error:0.08125
[3]	validation_0-error:0.08125
[4]	validation_0-error:0.08125
[5]	validation_0-error:0.08125
[6]	validation_0-error:0.08125
[7]	validation_0-error:0.08125
[8]	validation_0-error:0.08125
[9]	validation_0-error:0.08125
[10]	validation_0-error:0.08125
[11]	validation_0-error:0.08125
[12]	validation_0-error:0.08125
[13]	validation_0-error:0.08125
[14]	validation_0-error:0.08125
[15]	validation_0-error:0.08125
[16]	validation_0-error:0.08125
[17]	validation_0-error:0.08125
[18]	validation_0-error:0.08125
[19]	validation_0-error:0.08125
[20]	validation_0-error:0.08125
[21]	validation_0-error:0.08125
[22]	validation_0-error:0.08125
[23]	validation_0-error:0.08125
[24]	validation_0-error:0.08125
[25]	validation_0-error:0.08125
[26]	validation_0-error:0.08125
[27]	validation_0-error:0.08125
[28]	validation_0-error:0.08125
[29]	validatio

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.01,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=500, n_jobs=4, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [311]:
pre = my_model.predict(X_test)

In [312]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [313]:
print(confusion_matrix(Y_test,pre))
print("\n")
print(classification_report(Y_test,pre))

[[694   3]
 [ 33  70]]


              precision    recall  f1-score   support

           0       0.95      1.00      0.97       697
           1       0.96      0.68      0.80       103

   micro avg       0.95      0.95      0.95       800
   macro avg       0.96      0.84      0.89       800
weighted avg       0.96      0.95      0.95       800



In [314]:
test['churn']  = my_model.predict(test_1)

In [316]:
test['row_id'] = raw_test['row_id']

In [319]:
submit = test[['row_id','churn']]

In [321]:
submit.to_csv('heck.csv')